# Make a starting dataset with DFT
This notebook makes an initial dataset for training a model on dehyrdation by running molecular dynamics.
We have an initial structure from Huub and will run it at constant-temperature molecular dynamics.

In [1]:
from pathlib import Path
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.velocitydistribution import Stationary, ZeroRotation, MaxwellBoltzmannDistribution
from ase.calculators.socketio import SocketIOCalculator
from ase.calculators.nwchem import NWChem
from ase.io import read, iread
from ase.db import connect
from ase import units
import shutil

Configuration

In [2]:
temp_K = 298.15
run_dir = Path('md-run')
unixsocket = 'nwchem_ase'

Make a new dir

In [3]:
if run_dir.exists():
    shutil.rmtree(run_dir)
run_dir.mkdir()

## Load the initial structure
We have a molecule at the precipice of becoming dehydrated.

In [4]:
atoms = read('../example/stuff-oh-h.xyz')

## Create an NWChem Calculator
Ready to compute HF with "6-31g*" basis set

In [5]:
nwchem = NWChem(theory='scf', basis='6-31g*', task='optimize', directory=run_dir, driver={'socket': {'unix': unixsocket}})

## Perform Molecular Dynamics
Do NVT dynamics at at a temperature of 298.15K for 1000 timesteps, saving a frame at every step

First initialize the velocities

In [6]:
MaxwellBoltzmannDistribution(atoms, temperature_K=temp_K, force_temp=True)
Stationary(atoms)
ZeroRotation(atoms)
print(f'Initial temperature is {atoms.get_temperature():.2f}K')

Initial temperature is 298.15K


Set up the run

In [7]:
md = NVTBerendsen(atoms, timestep=10 * units.AUT,
                  temperature_K=temp_K, taut=0.2 * units.fs,
                  logfile=str(run_dir / 'md.log'),
                  trajectory=str(run_dir / 'md.traj'))

Run it

In [ ]:
with SocketIOCalculator(nwchem, unixsocket=unixsocket) as calc:
    atoms.calc = calc
    md.run(1000)

Convert to ASE DB, which we use as an interchange format

In [ ]:
with connect('initial-data.db', append=False) as db:
    for frame in iread(run_dir / 'md.traj'):
        db.write(frame)